In [ ]:
import json
import pandas as pd
import requests
import sqlalchemy
from datetime import datetime, date, timedelta
from pytz import timezone
import config



#FUNCTION with list of cities
def get_city_data(cities):
    API_key = config.API_key
    city_dict = {
            'city': [],
            'country_code': [],
            'population': [],
            'latitude': [],
            'longitude': [],
            'timezone': []
            # 'city_id': []
            }

    for j in range(len(cities)):
        weather_req = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={cities[j]}&appid={API_key}&units=metric")
        city_json = weather_req.json()
        city_dict['city'].append(city_json['city']['name'])
        city_dict['country_code'].append(city_json['city']['country'])
        city_dict['population'].append(city_json['city']['population'])
        city_dict['latitude'].append(city_json['city']['coord']['lat'])
        city_dict['longitude'].append(city_json['city']['coord']['lon'])
        city_dict['timezone'].append(city_json['city']['timezone'])
        # city_dict['city_id'].append(str(uuid.uuid4()))

    city_df = pd.DataFrame(city_dict)
    return city_df
        

#function that handles weather info
def get_weather_data(cities):
    # cities = ['Bishkek', 'Florence']
    API_key = config.API_key

    city_weather_dict = {
            'city': [],
            'country_code': [],
            'forecast_time': [],
            'outlook': [],
            'temp': [],
            'feels_like': [],
            'wind_speed': [],
            'humidity': []
            }

    for j in range(len(cities)):
        weather_req = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={cities[j]}&appid={API_key}&units=metric")
        city_weather_json = weather_req.json()

        for i in range(len(city_weather_json['list'])):
            city_weather_dict['city'].append(city_weather_json['city']['name'])
            city_weather_dict['country_code'].append(city_weather_json['city']['country'])
            city_weather_dict['forecast_time'].append(city_weather_json['list'][i]['dt_txt'])
            city_weather_dict['outlook'].append(city_weather_json['list'][i]['weather'][0]['description'])
            city_weather_dict['temp'].append(city_weather_json['list'][i]['main']['temp'])
            city_weather_dict['feels_like'].append(city_weather_json['list'][i]['main']['feels_like'])
            city_weather_dict['wind_speed'].append(city_weather_json['list'][i]['wind']['speed'])
            city_weather_dict['humidity'].append(city_weather_json['list'][i]['main']['humidity'])
        

    city_weather_df = pd.DataFrame(city_weather_dict)
    return city_weather_df


#lambda handler function    
def lambda_handler(event, context):
    # TODO implement
    schema="weather_pipelines"
    host= config.AWS_endpoint_url  
    user="admin"
    password=config.aws_password
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    # cities = get_city_data()['city'].to_list()
    cities = ['Bishkek', 'Istanbul', 'Florence']
    city_weather_df = get_weather_data(cities)
    city_df = get_city_data(cities)
    city_weather_df.to_sql('weather',con=con,if_exists='append',index=False)
    city_df.to_sql('cities',con=con,if_exists='replace',index=False)
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
